In [1]:
import os
import yaml
import pandas as pd
from pyathena import connect
from pyathena.util import as_pandas
from sqlalchemy import *

%matplotlib inline
%load_ext autoreload

In [2]:
pg_cred = yaml.load(open("../conf/local/credentials.yml"), Loader=yaml.FullLoader)

In [3]:
url = 'postgresql://{}:{}@{}:{}/{}'
url = url.format(pg_cred["pg_user"], pg_cred["pg_pass"], pg_cred["pg_host"], 5432, "iefp")

# The return value of create_engine() is our connection object
con = create_engine(url, client_encoding='utf8')

# We then bind the connection to MetaData()
meta = MetaData(bind=con, reflect=True)

/home/rosa_lavelle/IEFP-RecSys/env/lib/python3.6/site-packages/ipykernel_launcher.py:8: SADeprecationWarning: reflect=True is deprecate; please use the reflect() method.
  


In [4]:
# Interventions over time:

sql = """select ute_id, ano_mes, codigo_intervencao,

resultado_intervencao, dcodigo_interv 

from intervencoes

join tipos_intervencoes

on intervencoes.codigo_intervencao = tipos_intervencoes.codigo_interv

where ((intervencoes.tipo_movimento > 30) 

and intervencoes.codigo_intervencao != '0101' 
and intervencoes.codigo_intervencao != '0102')

order by ute_id

limit 10;
;"""

interv_time = pd.read_sql(sql, con)

In [13]:

sql = """select ute_id, ano_mes, codigo_intervencao, intervencoes.tipo_movimento, resultado_intervencao, dcodigo_interv 

from intervencoes

join tipos_intervencoes

on intervencoes.codigo_intervencao = tipos_intervencoes.codigo_interv

order by ute_id, ano_mes

limit 10;
;"""

interv = pd.read_sql(sql, con)

In [14]:
interv.head(10)

,ute_id,ano_mes,codigo_intervencao,tipo_movimento,resultado_intervencao,dcodigo_interv
0,0.0,201010.0,7003.0,25.0,1.0,ENTREVISTA COLOCAÇÃO
1,0.0,201010.0,7003.0,35.0,1.0,ENTREVISTA COLOCAÇÃO
2,0.0,201010.0,101.0,35.0,20.0,INICIO PPE
3,0.0,201010.0,101.0,25.0,20.0,INICIO PPE
4,0.0,201012.0,215.0,26.0,NaN,RVCC - BÁSICO - IEFP
5,0.0,201012.0,215.0,26.0,NaN,RVCC - BÁSICO - IEFP
6,2.0,201005.0,7011.0,25.0,1.0,RENDIMENTO SOCIAL DE INSERÇÃO - FIM
7,2.0,201005.0,7011.0,35.0,1.0,RENDIMENTO SOCIAL DE INSERÇÃO - FIM
8,28.0,201005.0,7011.0,25.0,1.0,RENDIMENTO SOCIAL DE INSERÇÃO - FIM
9,28.0,201005.0,7011.0,35.0,1.0,RENDIMENTO SOCIAL DE INSERÇÃO - FIM


In [7]:
interv_time.head(20)

,ute_id,ano_mes,codigo_intervencao,resultado_intervencao,dcodigo_interv
0,0.0,201010.0,7003.0,1.0,ENTREVISTA COLOCAÇÃO
1,2.0,201005.0,7011.0,1.0,RENDIMENTO SOCIAL DE INSERÇÃO - FIM
2,28.0,201005.0,7011.0,1.0,RENDIMENTO SOCIAL DE INSERÇÃO - FIM
3,38.0,201305.0,7003.0,1.0,ENTREVISTA COLOCAÇÃO
4,38.0,201308.0,772.0,24.0,EFA - S3 - TIPO A
5,38.0,201308.0,772.0,24.0,EFA - S3 - TIPO A
6,38.0,201309.0,7003.0,1.0,ENTREVISTA COLOCAÇÃO
7,38.0,201309.0,772.0,14.0,EFA - S3 - TIPO A
8,38.0,201309.0,772.0,14.0,EFA - S3 - TIPO A
9,38.0,201310.0,7003.0,1.0,ENTREVISTA COLOCAÇÃO


In [15]:
# count interventions per month where not PPE and not >30
sql = """select ano_mes, count(codigo_intervencao)

from intervencoes

where ((intervencoes.tipo_movimento > 30) 

and intervencoes.codigo_intervencao != '0101' 
and intervencoes.codigo_intervencao != '0102')

group by ano_mes

order by ano_mes

;"""

interv_time = pd.read_sql(sql, con)

In [9]:
interv_time.head(20)

,ano_mes,count
0,201002.0,148456
1,201003.0,178718
2,201004.0,161811
3,201005.0,298744
4,201006.0,148282
5,201007.0,188758
6,201008.0,142989
7,201009.0,189033
8,201010.0,163646
9,201011.0,167971


In [10]:
# count results interventions per month where not PPE and not >30
sql = """select ano_mes, count(codigo_intervencao)

from intervencoes

join tipos_intervencoes

on intervencoes.codigo_intervencao = tipos_intervencoes.codigo_interv

where ((intervencoes.tipo_movimento > 30) 

and intervencoes.codigo_intervencao != '0101' 
and intervencoes.codigo_intervencao != '0102')

order by ute_id

limit 10;
;"""

interv_time = pd.read_sql(sql, con)

ProgrammingError: (psycopg2.errors.GroupingError) column "intervencoes.ano_mes" must appear in the GROUP BY clause or be used in an aggregate function
LINE 1: select ano_mes, count(codigo_intervencao)
               ^
 [SQL: "select ano_mes, count(codigo_intervencao)\n\nfrom intervencoes\n\njoin tipos_intervencoes\n\non intervencoes.codigo_intervencao = tipos_intervencoes.codigo_interv\n\nwhere ((intervencoes.tipo_movimento > 30) \n\nand intervencoes.codigo_intervencao != '0101' \nand intervencoes.codigo_intervencao != '0102')\n\norder by ute_id\n\nlimit 10;\n;"] (Background on this error at: http://sqlalche.me/e/f405)